In [ ]:
import pandas as pd, pickle, numpy as np, warnings, time, os, re,pprint
from unidecode import unidecode
warnings.filterwarnings("ignore", "FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas")
pd.options.mode.copy_on_write = True
from config_path import PATH_CLEAN, PATH_API
from functions_shared import chunkify, work_csv, my_country_code, prop_string
from step7_persons.prep_persons import persons_preparation
# from step7_persons.affiliations import persons_affiliation
from api_process.openalex import get_author_from_openalex

CSV_DATE='20250121'
# persons_preparation(CSV_DATE)

PATH_PERSONS=f"{PATH_API}persons/"
perso_part = pd.read_pickle(f"{PATH_CLEAN}persons_participants.pkl")
perso_app = pd.read_pickle(f"{PATH_CLEAN}persons_applicants.pkl")




In [19]:
lvar=['project_id', 'generalPic', 'role', 
       'title_clean', 'gender', 'email', 'tel_clean', 'domaine_email',
       'orcid_id', 'birth_country_code', 'nationality_country_code',
       'host_country_code', 'sending_country_code', 'iso2', 'stage', 'contact',
       'country_code', 'shift', 'call_year', 'thema_code', 'destination_code',
       'entities_id', 'entities_name', 'id_secondaire', 'country_code_mapping']
pp = pd.concat([perso_part.drop_duplicates(), perso_app.drop_duplicates()], ignore_index=True)

mask=((pp.country_code=='FRA')|(pp.nationality_country_code=='FRA')|(pp.destination_code.isin(['COG', 'PF', 'STG', 'ADG', 'POC','SyG', 'PERA', 'SJI'])))&(~(pp.contact.isnull()&pp.orcid_id.isnull()))
pp=pp.loc[mask, lvar].sort_values(['country_code','orcid_id'], ascending=False).drop_duplicates()
pp['contact']=pp.contact.str.replace('-', ' ')

In [ ]:
def persons_files_import(thema, PATH_PERSONS):
    import re, os
    fname=''.join([filename for filename in os.listdir(PATH_PERSONS) if thema in filename])
    print(fname)

    if fname:
        with open(f"{PATH_PERSONS}{fname}", 'rb') as f:
            return pickle.load(f)

    if fname == []:
        fmax=max(int(os.path.splitext(filename)[0].split('_')[-1]) for filename in os.listdir(PATH_PERSONS) if re.search(r"persons_authors_[0-9]+",filename))
        if fmax:
            with open(f"{PATH_PERSONS}persons_authors_{fmax}.pkl", 'rb') as f:
                return pickle.load(f)

In [ ]:
import re, os
# fname=os.listdir(PATH_PERSONS)
fname=[os.path.splitext(filename)[0] for filename in os.listdir(PATH_PERSONS) if CSV_DATE in filename]
fmax=max(int(os.path.splitext(filename)[0].split('_')[-1]) for filename in os.listdir(PATH_PERSONS) if re.search(r"persons_authors_[0-9]+",filename))
print(fname)
pers_api=[]

# if fname:
#     for i in fname:
#         print(i)
#         with open(f"{PATH_PERSONS}{i}.pkl", 'rb') as f:
#             x = pickle.load(f)
#             if x == []:
#                 print(f"- empty list: {i}")
#             else:
#                 pers_api.extend(x)

# if fmax:
#     with open(f"{PATH_PERSONS}persons_authors_{fmax}.pkl", 'rb') as f:
#         x = pickle.load(f)
#         if x == []:
#             print(f"- empty list: {i}")
#         else:
#             pers_api.extend(x)

# print(len(pers_api))

# with open(f'{PATH_PERSONS}persons_authors_{CSV_DATE}.pkl', 'wb') as f:
#     pickle.dump(pers_api, f)

In [12]:
with open(f'{PATH_PERSONS}persons_authors_other_{CSV_DATE}.pkl', 'rb') as f:
    pers_api = pickle.load(f)

In [13]:
def persons_api_simplify(df):
    pers = [] 
    for p in df:
        # elem = {k: v for k, v in p.items() if (v and v != "NaT")}

        p['institutions'] = []
        if p.get("affiliations"):
            for aff in p["affiliations"]:  
                res={"institution_name":aff.get('institution').get("display_name"),
                "institution_ror":aff.get('institution').get("ror"),
                "institution_country":aff.get('institution').get("country_code"),
                "years":aff.get("years")}
                p['institutions'].append(res)
    
        p["orcid_openalex"] = p["ids"].get("orcid")            

        delete=['display_name_alternatives', 'topics', 'affiliations', 'id', 'last_known_institutions', 'ids']
        for field in delete:
            if p.get(field):
                p.pop(field)

        # elem = {k: v for k, v in elem.items() if (v and v != "NaT")}
        pers.append(p)

    print(len(pers))
    return pers

pers = persons_api_simplify(pers_api)

68961


In [ ]:
def persons_results_clean(df):
    from unidecode import unidecode
    from functions_shared import my_country_code, prop_string

    # df=pd.json_normalize(df, max_level=1)
    df=pd.json_normalize(df, record_path=['institutions'], meta=['match', 'orcid', 'display_name', 'orcid_openalex'], errors='ignore')
    df=df[~df.astype(str).duplicated()]
    cols = ['display_name']
    df = prop_string(df, cols)

    df['rows_by_name_orcid'] = df.groupby(['display_name', 'orcid'], dropna=False).transform('size')

    persName_withOrcid_noAff=df[(df.match=='full_name')&(~df.orcid.isnull())&(df.institution_name.isnull())]
    print(f"size person detect by name with an orcid but no affiliations: {len(persName_withOrcid_noAff)}")


    for i in ['orcid_openalex', 'orcid', 'institution_ror']:
        df.loc[~df[i].isnull(), i] = df.loc[~df[i].isnull()][i].str.split("/").str[-1]
    df['institution_ror'] = 'R'+ df['institution_ror'].astype(str)

    df['years']=df['years'].map(lambda liste: ';'.join(str(x) for x in liste))
    df=df[['display_name', 'orcid_openalex', 'years', 'institution_ror', 'institution_name', 'institution_country', 'rows_by_name_orcid']]
    my_countries=my_country_code()
    df=(df.merge(my_countries[['iso2', 'iso3', 'parent_iso3']].drop_duplicates(), 
                 how='left', left_on='institution_country', right_on='iso2')
        .drop(columns=['iso2'])
        )

    from step8_referentiels.paysage import paysage_prep
    from config_path import PATH
    DUMP_PATH=f'{PATH}referentiel/'
    paysage = paysage_prep(DUMP_PATH)
    df=(df.merge(paysage[['nom_long', 'numero_ror', 'numero_paysage', 'country_code_map', 'num_nat_struct']].drop_duplicates(), 
                 how='left', left_on='institution_ror', right_on='numero_ror'))



#     print(f"-3 size {match} cleaned: {len(df)}")
    return df

# pers_orcid=persons_results_clean(pers, 'orcid')
pers_oth=persons_results_clean(pers)


size person detect by name with an orcid but no affiliations: 0


c:\Users\zfriant\OneDrive\Github\pcri\functions_shared.py:255: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'AU' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df[c]==k, 'parent_iso2'] = v


- def(my_country_code) size df: 253
13409


In [25]:
oth=pp.loc[~pp.thema_code.isin(['ERC', 'MSCA'])].drop_duplicates().reset_index(drop=True).sort_values('contact')
oth

,project_id,generalPic,role,title_clean,gender,email,tel_clean,domaine_email,orcid_id,birth_country_code,...,contact,country_code,shift,call_year,thema_code,destination_code,entities_id,entities_name,id_secondaire,country_code_mapping
11037,101129622,999854758,fellow,NaN,non binary,NaN,NaN,,NaN,NaN,...,1 administrative europe engineer to be hired,FRA,NaN,2023,PATHFINDER,OPEN,8618D,École normale supérieure PSL,NaN,FRA
11042,101129622,999997833,fellow,NaN,non binary,NaN,NaN,,NaN,NaN,...,1 engineer to be hired,FRA,NaN,2023,PATHFINDER,OPEN,bZiTA,Institut national de la santé et de la recherc...,NaN,FRA
11032,101129622,896732021,fellow,NaN,non binary,NaN,NaN,,NaN,NaN,...,1 engineer to be hired,FRA,NaN,2023,PATHFINDER,OPEN,ieWd8,Immunrise,NaN,FRA
38428,101184939,999997930,main_contact,dr,male,aaron.boone@meteo.fr,0561079844,meteo,NaN,NaN,...,aaron boone,FRA,NaN,2024,CLUSTER 5-Climate,Destination 1 - D1,n2X5f,Centre national de la recherche scientifique,NaN,FRA
833,101189238,929945888,fellow,dr,male,NaN,NaN,,NaN,NaN,...,aaron cohen,ISR,NaN,2024,ACCELERATOR,OPEN,pic929945888,Spacepharma r&d israel ltd,NaN,ISR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19699,101058292,999997833,main_contact,prof,male,zsolt.lenkei@inserm.fr,0679634355,inserm,NaN,NaN,...,zsolt lenkei,FRA,NaN,2021,TRANSITION,OPEN,bZiTA,Institut national de la santé et de la recherc...,NaN,FRA
4719,101070670,999992401,fellow,dr,non binary,NaN,NaN,,NaN,NaN,...,zuber martin,FRA,NaN,2021,CLUSTER 3,Destination 4 - CS,zYXK8,Commissariat à l’énergie atomique et aux énerg...,NaN,FRA
17296,101187908,999992401,fellow,dr,female,NaN,NaN,,NaN,NaN,...,zuzana gerber,FRA,NaN,2024,INFRA,INFRADEV,zYXK8,Commissariat à l’énergie atomique et aux énerg...,NaN,FRA
22169,101071418,888824096,main_contact,dr,female,zuzanna@navalgo.com,0645442820,navalgo,NaN,NaN,...,zuzanna kosowska stamirowska,FRA,NaN,2021,SCALEUP,SCALEUP,86deo,Pathway,NaN,FRA


In [137]:
# pers_oth[pers_oth.display_name.str.contains('aubert')].display_name.unique()
df=oth.merge(pers_oth, how='inner', left_on=['contact', 'country_code'], right_on=['display_name','iso3'])
df=df[~df.astype(str).duplicated()]
df['years']=df['years'].map(lambda liste: ';'.join(str(x) for x in liste))
df['filt']=df.apply(lambda x: x['call_year'] in x['years'], axis=1)



#extract entities_id begun by PIC
work_csv(df.loc[df.entities_id.str.startswith('pic', na=False)], 'verif_id_struct_from openalex')

#keep only rows with th same year
df=df.loc[(df.filt==True)&(~df.num_nat_struct.isnull()), ['project_id', 'generalPic', 'role', 
       'nationality_country_code', 
        'stage', 'contact', 'display_name','country_code', 
        'entities_id', 'entities_name',
       'id_secondaire', 'country_code_mapping', 
        'institution_ror', 'institution_name',
       'institution_country',  'iso3', 'parent_iso3',
       'nom_long', 'numero_ror', 'numero_paysage', 'country_code_map',
       'num_nat_struct','rows_by_name_orcid','call_year','years', 'thema_code', 'destination_code']]
# 



In [138]:
df

,project_id,generalPic,role,nationality_country_code,stage,contact,display_name,country_code,entities_id,entities_name,...,nom_long,numero_ror,numero_paysage,country_code_map,num_nat_struct,rows_by_name_orcid,call_year,years,thema_code,destination_code
3,101184939,999997930,main_contact,NaN,evaluated,aaron boone,aaron boone,FRA,n2X5f,Centre national de la recherche scientifique,...,Centre national de recherches météorologiques,R004rej139,wP2cA,FRA,201320566C,10,2024,2024;2023;2022;2021;2020;2019;2018;2017;2016;2015,CLUSTER 5-Climate,Destination 1 - D1
70,101070157,999859608,main_contact,NaN,evaluated,abbas bradai,abbas bradai,FRA,hlX1r,Université de Poitiers,...,XLIM,R00f7srh09,5Lf14,FRA,201220195D;200612230X,10,2021,2022;2021;2020;2019;2018;2017;2015,CLUSTER 4-Digital,Destination 3 - DATA
82,101092664,999997930,main_contact,NaN,evaluated,abbas bradai,abbas bradai,FRA,n2X5f,Centre national de la recherche scientifique,...,XLIM,R00f7srh09,5Lf14,FRA,201220195D;200612230X,10,2022,2022;2021;2020;2019;2018;2017;2015,CLUSTER 4-Digital,Destination 3 - DATA
83,101092664,999997930,main_contact,NaN,evaluated,abbas bradai,abbas bradai,FRA,n2X5f,Centre national de la recherche scientifique,...,Institut p' : recherche et ingénierie en matér...,R05vjdsn22,Lg5It,FRA,201019365K,10,2022,2022,CLUSTER 4-Digital,Destination 3 - DATA
101,101071198,999997930,main_contact,NaN,evaluated,abdel mouttalib ouagazzal,abdel mouttalib ouagazzal,FRA,n2X5f,Centre national de la recherche scientifique,...,Laboratoire de neurosciences cognitives,R02dg3n954,6ikPc,FRA,201220268H,10,2021,2023;2021;2019;2018,PATHFINDER,CHALLENGES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201199,101060075,999993274,fellow,FRA,successful,zohra bouamra mechemache,zohra bouamra mechemache,FRA,t4SA4,Institut national de recherche pour l'agricult...,...,Fondation Jean-Jacques Laffont - Toulouse scie...,R00ff5f522,6fgha,FRA,200617702V,10,2021,2024;2023;2022;2021;2020;2019;2018;2017;2016;2015,CLUSTER 6,Destination 7 - GOVERNANCE
201216,101099649,888146648,main_contact,NaN,evaluated,zoltan takats,zoltan takats,FRA,U8a0v,Université de Lille,...,"Protéomique, réponse inflammatoire et spectrom...",R05dm3m993,8H32I,FRA,201119484K,10,2022,2024;2022;2021;2020,PATHFINDER,OPEN
201261,101172730,897556521,main_contact,NaN,evaluated,zoubir khatir,zoubir khatir,FRA,3Z5e6,Université Gustave Eiffel,...,Laboratoire des Systèmes et Applications des T...,R03vam5b06,g1w9e,FRA,200212702C,10,2024,2024;2023;2022;2021;2020;2019;2018;2017;2016;2015,CLUSTER 5-Energy,Destination 3 - D3
201270,101112295,897556521,main_contact,FRA,evaluated,zoubir khatir,zoubir khatir,FRA,3Z5e6,Université Gustave Eiffel,...,Laboratoire des Systèmes et Applications des T...,R03vam5b06,g1w9e,FRA,200212702C,10,2022,2024;2023;2022;2021;2020;2019;2018;2017;2016;2015,JU-JTI,Chips


In [81]:
# x.groupby(['project_id', 'generalPic', 'contact', 'country_code', 'stage']).filter(lambda y: y['entities_id'].isin(y.numero_paysage))
# x.groupby(['project_id', 'generalPic', 'contact', 'country_code', 'stage']).filter(lambda y: y['entities_id']==y['numero_paysage'])
x.columns
x.loc[x.years.str.contains(x.call_year.astype(str)), ['project_id', 'generalPic', 'role', 
       'nationality_country_code', 
        'stage', 'contact', 'display_name','country_code', 
        'entities_id', 'entities_name',
       'id_secondaire', 'country_code_mapping', 
        'institution_ror', 'institution_name',
       'institution_country',  'iso3', 'parent_iso3',
       'nom_long', 'numero_ror', 'numero_paysage', 'country_code_map',
       'num_nat_struct','rows_by_name_orcid','call_year','years', 'thema_code', 'destination_code']]

TypeError: unhashable type: 'Series'

In [ ]:
oth=pp.loc[~pp.thema_code.isin(['ERC', 'MSCA']), ['contact', 'orcid_id', 'iso2']].drop_duplicates().reset_index(drop=True)
oth[oth.contact.str.contains('aubert')].contact.unique()

In [ ]:
# def openalex_name(author):
#     import time, requests
#     try:
#         url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
#         nb_openalex=requests.get(url).json().get("meta").get('count')
#         dl=[]
#         if nb_openalex>0:
#             print(nb_openalex)
#             for n in range(0, nb_openalex): 
#                 author_openalex = requests.get(url).json().get("results")[n]
#                 if author_openalex.get('affiliations')!=[]:
#                     result=author | {'display_name':author_openalex.get('display_name'), 
#                                     'openalex_id':author_openalex.get('id'), 
#                                     'affiliations':author_openalex.get('affiliations'),
#                                     'topics':author_openalex.get('topics'),
#                                     'x_concepts':author_openalex.get('x_concepts'), 
#                                     'ids':author_openalex.get('ids'), 
#                                     'display_name_alternatives':author_openalex.get('display_name_alternatives'),
#                                     'match':'name'}
#                     dl.append(result)
#         return dl
    
#     # except requests.exceptions.HTTPError as http_err:
#     #     print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
#     except requests.exceptions.RequestException as err:
#         print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")    
#     except Exception as e:
#         print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")

# def openalex_orcid(author):
#     from config_api import openalex_usermail
#     import requests, time
#     try:
#         url = f"https://api.openalex.org/authors/orcid:{author.get('orcid')}?mailto={openalex_usermail}"
#         author_openalex = requests.get(url).json()
#         result = author | {'display_name':author_openalex.get('display_name'), 
#                            'openalex_id':author_openalex.get('id'), 
#                            'affiliations':author_openalex.get('affiliations'), 
#                            'topics':author_openalex.get('topics'),  
#                            'x_concepts':author_openalex.get('x_concepts'), 
#                            'ids':author_openalex.get('ids'), 
#                            'display_name_alternatives':author_openalex.get('display_name_alternatives'),
#                            'match':'orcid'}
#         return result
    
#     # except requests.exceptions.HTTPError as http_err:
#     #     print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
#     #     return author
#     except requests.exceptions.RequestException as err:
#         print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")
#         return author         
#     except Exception as e:
#         print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
#         return author
    


    

In [ ]:
# import time, pickle, requests
# from step7_persons.affiliations import openalex_name, openalex_orcid
# from config_path import PATH_CLEAN, PATH_API, PATH_WORK
# print(time.strftime("%H:%M:%S"))
# rlist=[]

# try:


#     author = {
#     "name": "caye pierre'",
#     "orcid": ""
#     }

#     if author.get("orcid"):
#         result = openalex_orcid(author)
#         if result.get('match'):
#             rlist.append(result)
#         else:
#             result = openalex_name(author)
#             if result:
#                 rlist.extend(result)
#     if author.get("orcid")=='':
#         result = openalex_name(author)
#         if result:
#             rlist.extend(result)

#     nf=f"persons_author"
#     with open(f'{PATH_WORK}test.pkl', 'wb') as f:
#         pickle.dump(rlist, f)
#     print(time.strftime("%H:%M:%S"))

# except requests.exceptions.HTTPError as http_err:
#     print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
#     nf=f"persons_author"
#     with open(f'{PATH_WORK}test.pkl', 'wb') as f:
#         pickle.dump(rlist, f)
#     print(time.strftime("%H:%M:%S"))

In [ ]:

with open(f'{PATH_WORK}test.pkl', 'rb') as f:
    test = pickle.load(f)
test

## return results from apenalex

1 - à partir d'orcid
2 - à partir du nom car orcid pas dans openalex -> vérifier que même personne ?
3 - à partir du nom car orcid non renseigné


In [ ]:
pers_api.mask(pers_api=='', inplace=True)
tmp1=pers_api.merge(my_countries, how='left', left_on='country_code', right_on='iso2')
# pers_api=country_clean(pers_api, ['country_code'])
tmp1=pers_api.loc[pers_api.match=='orcid']
len(tmp1)


In [ ]:
print(f"{len(perso_part)} ; {perso_part.columns}")

In [ ]:
tmp_part=perso_part.merge(tmp1, how='inner', left_on=['orcid_id'], right_on=['orcid'], suffixes=('','_api'))
tmp_part.loc[tmp_part.country_code!=tmp_part.country_code_api]

In [ ]:
data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000},
        ],
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337},
        ],
    },
]
data

In [ ]:
def openalex_name(author):
    url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
    nb_openalex=requests.get(url).json().get("meta").get('count')
    result={}
    if nb_openalex>0:
        for n in range(nb_openalex): 
            author_openalex = requests.get(url).json().get("results")[n]
            result.update({'display_name':author_openalex.get('display_name'), 'openalex_id':author_openalex.get('id'), 'affiliations':author_openalex.get('affiliations'), 'topics':author_openalex.get('topics'),  'x_concepts':author_openalex.get('x_concepts'), 'ids':author_openalex.get('ids'), 'display_name_alternatives':author_openalex.get('display_name_alternatives')})
            author.update(result)
            df=pd.concat([df, pd.json_normalize(author)])
    return df

In [ ]:
from config_path import PATH_API
import os, pandas as pd
 

for racine, repertoires, fichiers in os.walk(PATH_API):
    print(f"{racine}, {repertoires}, {fichiers}")
    for fichier in fichiers:
        if fichier.startswith('persons'):
            print(os.path.join(racine, fichier))
            globals()[f"{fichier}"]= pd.read_pickle(os.path.join(racine, fichier))


In [ ]:
fl=['persons_author.pkl', 'persons_author_10000.pkl', 'persons_author_12000.pkl', 'persons_author_2000.pkl', 'persons_author_4000.pkl', 'persons_author_6000.pkl', 'persons_author_8000.pkl', 'persons_author_name.pkl', 'persons_author_orcid.pkl']
for racine, repertoires, fichiers in os.walk(PATH_API):
    for i in fl:
        name=f"{i}".split('.')[0]
        print(name)
        globals()[name] = pd.read_pickle(os.path.join(racine, fichier))

In [ ]:
pers=pd.concat([persons_author_10000, persons_author_12000, persons_author_2000, persons_author_4000, persons_author_6000, persons_author_8000], ignore_index=True)

fl=['persons_author.pkl', 'persons_author_name.pkl', 'persons_author_orcid.pkl']

In [ ]:
def openalex_orcid(author):
    # from config_api import openalex_usermail
    import requests, time
    try:
        url = f"https://api.openalex.org/authors/orcid:{author.get('orcid')}?mailto=zmenesr@gmail.com"
        author_openalex = requests.get(url).json()
        result = author | {'display_name':author_openalex.get('display_name'), 
                           'openalex_id':author_openalex.get('id'), 
                           'affiliations':author_openalex.get('affiliations'), 
                           'topics':author_openalex.get('topics'),  
                           'x_concepts':author_openalex.get('x_concepts'), 
                           'ids':author_openalex.get('ids'), 
                           'display_name_alternatives':author_openalex.get('display_name_alternatives'),
                           'match':'orcid'}
        return result
    
    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
        return author
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")
        return author           
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
        return author

In [ ]:
def openalex_name(author):
    import time
    try:
        url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
        nb_openalex=requests.get(url).json().get("meta").get('count')
        dl=[]
        if nb_openalex>0:
            print(nb_openalex)
            for n in range(nb_openalex): 
                author_openalex = requests.get(url).json().get("results")[n]
                if author_openalex.get('affiliations')!=[]:
                    result=author | {'display_name':author_openalex.get('display_name'), 
                                    'openalex_id':author_openalex.get('id'), 
                                    'affiliations':author_openalex.get('affiliations'),
                                    'topics':author_openalex.get('topics'),
                                    'x_concepts':author_openalex.get('x_concepts'), 
                                    'ids':author_openalex.get('ids'), 
                                    'display_name_alternatives':author_openalex.get('display_name_alternatives'),
                                    'match':'name'}
                    dl.append(result)
        return dl
    
    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
        return author
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")
        return author           
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
        return author
    

In [ ]:
from config_path import PATH_CLEAN
# from functions_shared import chunkify
perso_part = pd.read_pickle(f"{PATH_CLEAN}persons_participants.pkl")
# pp = pd.concat([perso_part[['contact', 'orcid_id', 'country_code']].drop_duplicates(), perso_app[['contact', 'orcid_id', 'country_code']].drop_duplicates()], ignore_index=True)
pp = perso_part[['contact', 'orcid_id', 'country_code']].fillna('')
pp=pp.loc[(pp.country_code=='FRA')].sort_values('orcid_id', ascending=False)
pp
data_chunks=list(chunkify(pp, 10000))
for i in range(0, len(data_chunks)):
    print(f"Loop {i}, size data_chunks: {len(data_chunks)}")
    # print(type(data_chunks))
    df_temp = data_chunks[i]
    

In [ ]:
import pickle, pandas as pd
from config_path import PATH_API
pers_api=[]
for i in range(1,3):
    print(i)
    with open(f"{PATH_API}fr_persons_author_{i}.pkl", 'rb') as f:
        globals()[f"pers_api{i}"] = pickle.load(f)
    pers_api.extend(globals()[f"pers_api{i}"])


In [ ]:
pers_api=pd.json_normalize(pers_api1, record_path=['affiliations'], meta=['name', 'orcid', 'display_name', 'openalex_id',  'match',  ["ids", "orcid"]],
        errors='ignore')

pers_api.columns = pers_api.columns.str.replace(r"[.*_]+", '_', regex=True)

pers_api = (pers_api
            .rename(columns={
                    'institution_country_code':'country_code'})
            .drop(columns=['institution_type','institution_lineage']))

for i in ['ids_orcid', 'institution_ror']:
    pers_api.loc[~pers_api[i].isnull(), i] = pers_api.loc[~pers_api[i].isnull()][i].str.split("/").str[-1]

In [ ]:
from config_path import PATH_CLEAN
perso_part = pd.read_pickle(f"{PATH_CLEAN}persons_participants.pkl")
perso_app = pd.read_pickle(f"{PATH_CLEAN}persons_applicants.pkl")

In [ ]:
with open(f"{PATH_API}persons_author.pkl", 'rb') as f:
    author_orcid = pickle.load(f)

In [ ]:
author_orcid=pd.json_normalize(author_orcid, record_path=['affiliations'], meta=['name','orcid', 'display_name', 'ids', 'match'])
author_orcid